<a href="https://colab.research.google.com/github/sakuronohana/cas_datenanalyse/blob/master/Semesterarbeit/CAS_DA_SA_Payment_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img 
src="https://www.ffhs.ch/templates/ffhs/img/logo@2x.png" width="100"> 
###DaAn, Data Analysis, MAS/CAS Web4B 2018, ZH1, FS19, Dr. Tödtli Beat###

##*Semesterarbeit von Patrik Di Lena*##
#Betrugserkennung Zahlungsverkehr#



##Ziel##
Auf der Grundlage von maschinellen Lernalgorithmen soll ein Betrugserkennungssystem für den Zahlungsverkehr einer ausschliesslich in der Schweiz tätigen Bank erstellt werden. Auf Basis historisierter Daten in Bezug auf das E-Banking Session- und Zahlungsverhalten von Bankkunden sollen mögliche Betrugsintensionen erkannt und für bestimmte Risikogruppen vorhergesagt werden.

##Aufgabenstellung##

Aus dem oben formulierten Ziel ergeben sich zwei wesentliche Aufgabenstellungen:
1.	Erkennung betrügerischer Handlungen im Zahlungsverkehr
2.	Ermittlung potenzieller Kundenrisikogruppen

##Betrugsanfällige Risikogruppen##
Die für den Betrug im Zahlungsverkehr anfälligen Gruppen werden auf Basis der folgenden Merkmale durch den Lernalgorithmus bestimmt:

*	Geschlecht
*	Alter
*	Zivilstand
*	Nationalität
*	Kontosaldo
*	Wohnort
*	Region
*	Zahlungsmethode (E-Rechnung, Einzahlungsschein etc.)
*	Rechtsform (Juristisch oder natürliche Person)

## Datensatz##
Die im Rahmen dieser Semesterarbeit verwendeten Daten entstammen, von ihrer Datenstruktur her, aus einem bereits eingesetzten nicht ML-basierten E-Banking Betrugserkennungssystem. Die Datenwerte wurde zu Wahrung der datenschutzrechtlichen und bankengesetzlichen Aspekte mit Hilfe des folgenden Python-Scripts künstlich erzeugt und beinhalten somit keine reale Zahlungs-, Kunden-, und betrugsspezifischen Informationen. Weiter wurden einige durch die Bank definierten Spezifikationen bzw. Merkmale aus sicherheitstechnischen Gründen umbenannt.  

Datenselektion:

- 5'000 synthetische Kunden (Vorname, Name, Strasse, Ort, Kanton, Sprachregion, Geschlecht, Alter, Zivilstand,  Nationalität, Kontonummer, Vertragsnummer, Rechtsform )

- 100'000 synthetische Zahlungen (Transaktionsid, ,Erstellungdatum/Zeit, Empfängerkonto, Empfängerbank, Empfängerland, Währung, Betrag, Valuta Datum)








###Datengeneratoren###

Während die Stammdaten hauptsächlich im Excel manuell von verschiedenen Quellen 

Datenquellen:

Open Government Data https://opendata.swiss/de/


###IBAN Generator###

Mit der folgenden Code Cell wurden die IBAN Nummern für die Kunden Stammdaten erstellt. 

In [0]:
# IBAN Generator

import datetime
import numpy as np
import pandas as pd

# Datenquellen Github
ibandb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv'

# IBAN Generator in elektronischer Form

iban_cc = pd.read_csv(ibandb_url, delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','Sample','Bank']

# IBAN Gen. 1.Teil - Erstellung IBAN Länder- und Bankencode
code = np.random.choice(np.ravel(iban_cc['Code']).astype(str))
checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))
iban_country_code = code + checkdig
iban_bank_code = np.random.choice((iban_cc.loc[iban_cc['Code'] == code])['Length']).astype(str)
iban_ccbc = iban_country_code + "00" + iban_bank_code 

# IBAN Gen. 2.Teil - Erstellung Kontonummer und Zusammensetzung der IBAN
eiban = []
for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_ccbc + part_eiban.astype(str)

# Zufällige Auswahl einer IBAN aus dem generierten Datensatz
eiban = np.ravel(eiban)
eiban = np.random.choice(eiban)



LC7200322800994985921


###Payment Generator###

Mit dem folgenden Python Code wurden synthetischen Transaktionsdaten inkl. Betrugserkennungsidentifikation generiert. Die Daten wurde vollumfänglich Zufrallsgeneriert. Die Daten beinhalten folgenden Merkmale:

* **Zeitstempel** (timestamp) - Zeitpunkt der Transaktionserstellung
* **Zahlungsidentifikationsnummer** (paym_id) - Eindeutige Nummer pro erstellter Transaktion
* **Zahlungssendende Bank** (send_bc_nr) - Eindeutiger Bankcode der Bank, von welcher die zahlung gesendet wurde. In diesem Fall ist es immer die Gleiche
* **Zahlungsbetrag** (amount) - Überweisungsbetrag an das Empfängerkonto (rcv_iban)
* 
,rcv_bc_nr,rcv_bc_iban,rcv_bc_country,rcv_bc_code,rcv_iban,valuta_date,cust_vertrag_nr,cust_konto_nr,cust_vname,cust_nname,cust_plz,cust_ort,cust_kanton,cust_sprachregion,cust_auth_device,fraud_id



In [4]:
# Erstellung von Transaktionsdaten 

from datetime import datetime, timedelta
import numpy as np
import pandas as pd

# Bankbeziehungsdaten Datenquellen Github
ibandb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv'

# Transaktionsinfromationen importieren

iban_cc = pd.read_csv(ibandb_url, delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','IBAN','Bank']

# Betrugseinstufung 
# Kein Fraud ist immer 0
fraud_ids = [[1,'External Caused Fraud (e.g. Trojans)'],[2,'Internal Caused Fraud (e.g. Employee)']]
df_fraud = pd.DataFrame(fraud_ids, columns=['Fraud ID', 'Description'])

# Betragswaehrungen (Schweizer Franken, Euro, US-Dollar)
currency = [['CHF','Schweizer Franken'],['EUR','EU Euro'],['USD','US Dollar']]
df_currency = pd.DataFrame(currency, columns=['Currency', 'Description'])

# Bankkunden Stammdaten - Datenquellen Github
custdb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/cutomer_base_data.csv'

cust_data = pd.read_csv(custdb_url, delimiter=';')
cust_data.columns=['anrede','vorname','nachname','strasse','nr','plz','ort',
                   'kanton','sprachregion',	'nation',	'sexcode','mobile_nr',	
                   'vertrag_nr','konto_nr','auth_device']

# Importierte Kundendaten bereinigen
cust_data = cust_data.fillna(0)
cust_data_dtype = {'plz':int,'mobile_nr':int,'vertrag_nr':int}
cust_data = cust_data.astype(cust_data_dtype)

# Erstellung der Zahlungs-Basisinformationen 

# Erster Teil der Zahlungs-Basisinformationen

trx_data = pd.DataFrame(columns=['timestamp','paym_id','send_bc_nr','amount',
                                 'rcv_bc_nr','rcv_bc_iban','rcv_bc_country','rcv_bc_code',
                                 'rcv_iban','valuta_date','cust_vertrag_nr','cust_konto_nr',
                                 'cust_vname','cust_nname','cust_plz','cust_ort',
                                'cust_kanton','cust_sprachregion','cust_auth_device',
                                'fraud_id'])

for i in range(250):
    trx_data.loc[i,['timestamp']] = timestamps = pd.Timestamp(datetime(2018,1,1,8,0,0)+timedelta(seconds=30*i)).strftime("%Y.%m.%d %H:%M")
    trx_data.loc[i,['paym_id']] =  10000000 + 10 +i
    #trx_data.loc[i,['paym_id']] =  np.int64(np.random.randint(1000000000000,2000000000000,1))
    trx_data.loc[i,['send_bc_nr']] =  'CHBANK71XXX'
    trx_data.loc[i,['amount']] = np.int64(np.random.randint(100,100000,1)).astype(str)
    trx_data.loc[i,['rcv_bc_nr']] = bc_bank = np.random.choice(np.ravel(iban_cc['Bank']).astype(str))
    trx_data.loc[i,['rcv_bc_iban']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['IBAN']].astype(str))
    trx_data.loc[i,['rcv_bc_country']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Country']])
    trx_data.loc[i,['rcv_bc_code']] = bc_code = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Code']].astype(str))
    iban_ccbc = bc_code + np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Checkdig']].astype(str)) + '00'
    part_eiban = np.ravel(np.int64(np.random.randint(2000000000000,3000000000000,1)))
    trx_data.loc[i,['rcv_iban']] = iban_ccbc + part_eiban.astype(str)
    timestamps = datetime.strptime(timestamps,"%Y.%m.%d %H:%M")
    trx_data.loc[i,['valuta_date']] = valuta = datetime.strftime((timestamps + timedelta(hours=12*i)),'%Y.%m.%d %H:%M')
    trx_data.loc[i,['cust_vertrag_nr']] = cd_contract = np.random.choice(np.ravel(cust_data['vertrag_nr']).astype(int))
    trx_data.loc[i,['fraud_id']] = np.random.choice(np.ravel(df_fraud['Fraud ID']).astype(str)) 
# Zeiter Teil der Zahlungs-Basisinformationen. 
    for row in trx_data['cust_vertrag_nr']:
      if row == cd_contract:
        trx_data.cust_konto_nr = cust_data.konto_nr
      if row == cd_contract:
        trx_data.cust_vname = cust_data.vorname
      if row == cd_contract:
        trx_data.cust_nname = cust_data.nachname
      if row == cd_contract:
        trx_data.cust_plz = (cust_data.plz).astype(int)
      if row == cd_contract:
        trx_data.cust_ort = cust_data.ort
      if row == cd_contract:
        trx_data.cust_kanton = cust_data.kanton
      if row == cd_contract:
        trx_data.cust_sprachregion = cust_data.sprachregion
      if row == cd_contract:
        trx_data.cust_auth_device = cust_data.auth_device
     
       

# Von allen durchgeführten Zahlungen hat das Fraud-Detection System 0.5% Betrugszahlungen erkannt. 
trx_data_non_fraud = trx_data.sample(frac=0.95)

for x in range(len(trx_data_non_fraud)):
  trx_data_non_fraud.fraud_id = 0

  
# Löschung der Non Fraud aus DataFrame "Trx_Data" und Erstellung Fraud

trx_data_fraud = trx_data.drop(trx_data.index[trx_data_non_fraud.index])

# Non Fraud und Fraud Dataframes in ein csv exportieren
trx_data = pd.concat([trx_data_non_fraud, trx_data_fraud], axis=0)
trx_data.sort_values(['paym_id'], axis=0, ascending=True, inplace=True)
trx_data.to_csv('trx_data.csv', index=False)
trx_data.head(-5)
  

,timestamp,paym_id,send_bc_nr,amount,rcv_bc_nr,rcv_bc_iban,rcv_bc_country,rcv_bc_code,rcv_iban,valuta_date,cust_vertrag_nr,cust_konto_nr,cust_vname,cust_nname,cust_plz,cust_ort,cust_kanton,cust_sprachregion,cust_auth_device,fraud_id
0,2018.01.01 08:00,10000010.0,CHBANK71XXX,71420,ESBANK76XXX,ES1321000555370200853027,Spanien,ES,ES76002023750126601,2018.01.01 08:00,82902.0,CH5300772672529640195,Martin,Bader,6020,Emmenbrücke,LU,de,mtan,0
1,2018.01.01 08:00,10000011.0,CHBANK71XXX,80046,PSBANK62XXX,PS06ARAB000000009040781605610,Palästina,PS,PS62002271143817067,2018.01.01 20:00,80935.0,CH5300772670061796126,Margaretha,Fehr,9056,Gais,AR,de,ptan,0
2,2018.01.01 08:01,10000012.0,CHBANK71XXX,32103,ALBANK11XXX,AL06202111090000000005012075,Albanien,AL,AL11002816184518873,2018.01.02 08:01,84710.0,CH5300772500037725234,Daniel,Miéville,1253,Vandoeuvres,GE,fr,ptan,0
3,2018.01.01 08:01,10000013.0,CHBANK71XXX,95207,BABANK16XXX,BA391011606058553319,Bosnien und Herzegowina,BA,BA16002147474559195,2018.01.02 20:01,67250.0,CH5300772732508635138,Andrea,Iseni,6026,Rain,LU,de,mtan,0
4,2018.01.01 08:02,10000014.0,CHBANK71XXX,14827,LVBANK46XXX,LV02HABA0551007820897,Lettland,LV,LV46002252799877458,2018.01.03 08:02,75197.0,CH5300772640237288398,Andrea,Pinana,6693,Broglio,TI,it,mtan,0
5,2018.01.01 08:02,10000015.0,CHBANK71XXX,59600,GEBANK29XXX,GE02TB7523045063700002,Georgia,GE,GE29002982643944177,2018.01.03 20:02,67636.0,CH5300772172038822637,Peter,Häner,6006,Luzern,LU,de,ptan,0
6,2018.01.01 08:03,10000016.0,CHBANK71XXX,81316,NLBANK58XXX,NL02ABNA0457180536,Niederlande,NL,NL58002258191248613,2018.01.04 08:03,66945.0,CH5300772262693101243,Ursula,Derungs,6247,Schötz,LU,de,mtan,0
7,2018.01.01 08:03,10000017.0,CHBANK71XXX,23527,KZBANK41XXX,KZ04319C010005569698,Kasachstan,KZ,KZ41002360918983284,2018.01.04 20:03,78321.0,CH5300772511216015083,Michel,Kaenel,1665,Estavannens,FR,fr,ptan,0
8,2018.01.01 08:04,10000018.0,CHBANK71XXX,390,MTBANK51XXX,MT02VALL22013000000040013752732,Malta,MT,MT51002165803971540,2018.01.05 08:04,81901.0,CH5300772960799401160,Francesco,Lafranchi,6943,Vezia,TI,it,ptan,0
9,2018.01.01 08:04,10000019.0,CHBANK71XXX,58463,MUBANK53XXX,MU03MCBL0901000001879025000USD,Mauritius,MU,MU53002331664794367,2018.01.05 20:04,81219.0,CH5300772221245992471,Thomas,Müller,6020,Emmenbrücke,LU,de,ptan,0


#Ermittlung potenzieller Kundenrisikogruppen#


In [0]:
import pandas as pd
import numpy as np


# Laden der Transaktionsdaten 
trx_data_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/trx_data.csv'

trx_data = pd.read_csv(trx_data_url)
trx_data.head()